In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import tqdm

# Authenticate and create the PyDrive client.
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create a service object
drive_service = build('drive', 'v3')

# Download file from Google Drive
file_id = '1lhAaeQCmk2y440PmagA0KmIVBIysVMwu'

# Create a progress bar
with tqdm.tqdm(unit='B', unit_scale=True, miniters=1) as t:
    # Create a MediaIoBaseDownload object
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO('tennis_court_det_dataset.zip', mode='wb')
    downloader = MediaIoBaseDownload(fh, request)

    # Download the file with progress bar
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        if status:
            t.update(status.total_size - t.n)

# Unzip the file
!unzip /content/tennis_court_det_dataset.zip -d /content/

7.26GB [05:00, 24.1MB/s]


Streaming output truncated to the last 5000 lines.
  inflating: /content/data/images/JNKp7sCdQlY_2200.png  
  inflating: /content/data/images/JNKp7sCdQlY_2250.png  
  inflating: /content/data/images/JNKp7sCdQlY_250.png  
  inflating: /content/data/images/JNKp7sCdQlY_300.png  
  inflating: /content/data/images/JNKp7sCdQlY_350.png  
  inflating: /content/data/images/JNKp7sCdQlY_400.png  
  inflating: /content/data/images/JNKp7sCdQlY_450.png  
  inflating: /content/data/images/JNKp7sCdQlY_50.png  
  inflating: /content/data/images/JNKp7sCdQlY_500.png  
  inflating: /content/data/images/JNKp7sCdQlY_550.png  
  inflating: /content/data/images/JNKp7sCdQlY_600.png  
  inflating: /content/data/images/JNKp7sCdQlY_650.png  
  inflating: /content/data/images/JNKp7sCdQlY_700.png  
  inflating: /content/data/images/JNKp7sCdQlY_750.png  
  inflating: /content/data/images/juXbdW7z0WA_100.png  
  inflating: /content/data/images/juXbdW7z0WA_1050.png  
  inflating: /content/data/images/juXbdW7z0WA_1100.

# Start Code

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class KeyPointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w   # adjust x coordinates
        kps[1::2] *= 224.0 / h  # adjust y coordinates

        return img, kps


In [ ]:
train_dataset = KeyPointsDataset("data/images", "data/data_train.json")
val_dataset = KeyPointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [ ]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 159MB/s]


In [ ]:
model = model.to(device)

# Train model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14004.3076171875
Epoch 0, iter 10, loss: 14528.337890625
Epoch 0, iter 20, loss: 13683.74609375
Epoch 0, iter 30, loss: 13302.8466796875
Epoch 0, iter 40, loss: 13410.5654296875
Epoch 0, iter 50, loss: 13424.1962890625
Epoch 0, iter 60, loss: 12814.537109375
Epoch 0, iter 70, loss: 12160.806640625
Epoch 0, iter 80, loss: 12060.08984375
Epoch 0, iter 90, loss: 11450.3115234375
Epoch 0, iter 100, loss: 10902.341796875
Epoch 0, iter 110, loss: 10902.322265625
Epoch 0, iter 120, loss: 10389.9111328125
Epoch 0, iter 130, loss: 10001.1826171875
Epoch 0, iter 140, loss: 10801.197265625
Epoch 0, iter 150, loss: 9543.244140625
Epoch 0, iter 160, loss: 8784.470703125
Epoch 0, iter 170, loss: 8374.1552734375
Epoch 0, iter 180, loss: 8585.6787109375
Epoch 0, iter 190, loss: 7885.79931640625
Epoch 0, iter 200, loss: 8411.0068359375
Epoch 0, iter 210, loss: 7480.08984375
Epoch 0, iter 220, loss: 6895.47021484375
Epoch 0, iter 230, loss: 6786.3955078125
Epoch 0, iter 240, loss:

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")